# Can a model be written to win FiveThirtyEight's NFL prediction game?

The National Football League is one of the most watched sports leagues in the world, second to perhaps only Soccer. The Super Bowl (the championship of American Football) is the most watched sporting event year after year. Because of it's popularity, fans flock to fantasy football as well as gambling to further enhance their viewing experience. A popular sports and politics journalism site, FiveThrityEight, uses data driven approaches to try and forecast the outcome of each game. In addition, they allow their fans to take part and see how their predictions would stack up against FiveThirtyEight's model.

The way the game works is by assigning probabilities rather than just winners and losers. The system FiveThirtyEight uses scores each player's probability predictions. A correct prediction can net the player up to +25 points whereas an incorrect prediction can cost the player -75 points. The amount of points is determine by the Brier Score (https://en.wikipedia.org/wiki/Brier_score), but the essential thing to understand is that the system punishes aggresive votes for being wrong (i.e. if I say the probability of the home team winning is 91% and they lose, I would lose the majority of the roughly -58 points)and also punishes probability predictions that are too conservative (i.e. if I say the probability of the home team winning is 76% and they do, I would only gain 19 points). FiveThirtyEight's own model is based off of the Elo rating system (https://en.wikipedia.org/wiki/Elo_rating_system) and tends to be provide more conservative predictions. The full rules of the game can be found at (https://fivethirtyeight.com/features/how-to-play-our-nfl-predictions-game/) and a description of FiveThirtyEight's model is at (https://fivethirtyeight.com/features/introducing-nfl-elo-ratings/)

This series of notebooks will go through our process of creating a model to try and outperform FiveThirtyEight's own Elo model. In order to do this, we're going to incorporate NFL betting data, weather data, and FiveThrityEight's own Elo data. We'll walk through the process of data cleaning and preparation, do some light univariate and multivariate analysis, and then build a decision tree model. These notebooks will walk through each of those steps to ensure that everything is repeatable.

## Step 1: Data Loading & Preparation

The first step will be loading the data. We have chosen to include the CSVs from our various sources in the same directory as our Jupyter notebooks. You may want to instead read the files directly from a hyperlink. If so, you'll need to change the second cell below.

Each cell should be run in order once the data has been loaded. Each code segment is commented so that it explains what it does. Any major code segments will have a markdown cell explaining it further.

In [1]:
#short list of packages we'll be using
packages <- c("ggplot2","RColorBrewer","ggthemes","gridExtra","lubridate","tidyr","plyr","dplyr","reshape2","dummies","caTools","rpart","rpart.plot","rattle")

#install packages if they aren't already installed
#then load the packages
for (package in packages){
    if(!require(package,character.only=TRUE)) install.packages(package,character.only=TRUE)
    library(package,character.only=TRUE,quietly=TRUE)
}

Loading required package: ggplot2
Loading required package: RColorBrewer
Loading required package: ggthemes
Loading required package: gridExtra
Loading required package: lubridate

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Loading required package: tidyr
Loading required package: plyr

Attaching package: ‘plyr’

The following object is masked from ‘package:lubridate’:

    here

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union

The following object is masked from ‘package:gridExtra’:

    combine

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: reshape2



In [2]:
# Read in data sets

#sourced from:
#https://github.com/fivethirtyeight/nfl-elo-game
nfl_elo_df <- read.csv("nfl_games.csv", as.is = TRUE, header = TRUE)

#sourced from: 
#https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data
nfl_teams_df <- read.csv("nfl_teams.csv", as.is = TRUE, header = TRUE)
nfl_stadiums_df <- read.csv("nfl_stadiums.csv", as.is = TRUE, header = TRUE)
nfl_games_df <- read.csv("spreadspoke_scores.csv", as.is = TRUE, header = TRUE)

#we chose to download the data into the same folder as our notebooks, but you may want to change that and read directly
#from a web link

In [3]:
#some data prep

#transform dates so we can join the data easier
nfl_games_df$schedule_date <- as.Date(parse_date_time(nfl_games_df$schedule_date, "mdy"))
nfl_elo_df$date <- as.Date(parse_date_time(nfl_elo_df$date, "ymd"))

#join team data to games data
#this helps us remap some of the IDs and such so that all teams are identified the same way
nfl_df <- left_join(nfl_games_df,nfl_teams_df,by=c("team_home"="team_name"))
nfl_df <- left_join(nfl_df, nfl_teams_df, by = c("team_away" = "team_name"))

#rename variables to home and away
#after the join we get an x and y for matching variable names from the above joins
#this just remaps things so we can 
names(nfl_df)[names(nfl_df) == 'team_id.x'] <- 'home_team_id'
names(nfl_df)[names(nfl_df) == 'team_id.y'] <- 'away_team_id'
names(nfl_df)[names(nfl_df) == 'team_conference.x'] <- 'home_team_conference'
names(nfl_df)[names(nfl_df) == 'team_division.x'] <- 'home_team_division'
names(nfl_df)[names(nfl_df) == 'team_conference.y'] <- 'away_team_conference'
names(nfl_df)[names(nfl_df) == 'team_division.y'] <- 'away_team_division'
names(nfl_df)[names(nfl_df) == 'team_home'] <- 'home_team'
names(nfl_df)[names(nfl_df) == 'score_home'] <- 'home_score'
names(nfl_df)[names(nfl_df) == 'score_away'] <- 'away_score'
names(nfl_df)[names(nfl_df) == 'team_away'] <- 'away_team'

#join elo data to joined data
#the elo data and betting data has no common ID so we join on a few columns
nfl_df <- left_join(nfl_df, nfl_elo_df, by = c("schedule_date" = "date", "home_team_id" = "team1", "away_team_id" = "team2"))

#rename the elo columns to be more descriptive
names(nfl_df)[names(nfl_df) == 'elo1'] <- 'home_team_elo'
names(nfl_df)[names(nfl_df) == 'elo2'] <- 'away_team_elo'
names(nfl_df)[names(nfl_df) == 'elo_prob1'] <- 'home_team_win_prob'

#create even smaller dataframe in order to evaluate ELO and Spread effectiveness
nfl_df_filtered <- nfl_df[,c('schedule_date',
                             'schedule_season',
                             'home_team_id',
                             'away_team_id',
                             'home_score',
                             'away_score',
                             'team_favorite_id',
                             'spread_favorite',
                             'home_team_elo',
                             'away_team_elo',
                             'home_team_win_prob'
                            )]

#looking at the structure of our final dataframe
str(nfl_df_filtered)

'data.frame':	12411 obs. of  11 variables:
 $ schedule_date     : Date, format: "1966-09-02" "1966-09-03" ...
 $ schedule_season   : int  1966 1966 1966 1966 1966 1966 1966 1966 1966 1966 ...
 $ home_team_id      : chr  "MIA" "TEN" "LAC" "MIA" ...
 $ away_team_id      : chr  "OAK" "DEN" "BUF" "NYJ" ...
 $ home_score        : int  14 45 27 14 24 31 24 14 20 14 ...
 $ away_score        : int  23 7 7 19 3 0 0 19 42 3 ...
 $ team_favorite_id  : chr  "" "" "" "" ...
 $ spread_favorite   : num  NA NA NA NA NA NA NA NA NA NA ...
 $ home_team_elo     : num  1300 1377 1543 1288 1618 ...
 $ away_team_elo     : num  1523 1383 1619 1454 1587 ...
 $ home_team_win_prob: num  0.287 0.585 0.484 0.359 0.635 ...


## Step 2: Data Cleaning

Now that the data has been loaded variables selected and renamed, we can start cleaning our data. There will be several steps we'll need to take in order to ensure that our data is in the right format for our model. Firstly, we'll need to look at how much data we have an how much is incomplete.

### Addressing NULL values

Since are data comes from a few different sources (especially the Kaggle data) there are several nulls within our vairables. We can run a quick check on our filtered down dataset to see how many we have.

In [4]:
#checking nulls
print("Total NULLs in each column")
colSums(is.na(nfl_df_filtered))

[1] "Total NULLs in each column"


schedule_date    schedule_season       home_team_id       away_team_id 
                 0                  0                  0                  0 
        home_score         away_score   team_favorite_id    spread_favorite 
                 0                  0                  0               2479 
     home_team_elo      away_team_elo home_team_win_prob 
              1149               1149               1149

### Superbowl Era vs Modern Era 

One way that we can address this is by filtering the timeframe of our data. Our Kaggle data includes data from 1966 until 2018 whereas our FiveThirtyEight Elo data includes data from 1920 until 2017. Both these data sets cover multiple eras of the NFL during which the game was significantly different. For some background, I'll describe the two most common eras:

**The Superbowl Era** is the time after the NFL and AFL (American Football League) merged. These two leagues were both fairly large and merged into one league between 1966 and 1970 with the first post season merger happening in 1971. From 1971 one there were two conferences (the AFC and NFC) and the two best teams from each conference played each other in the Superbowl (hence the name). This era is defined as 1971 to the present.

**The Modern Era** is the time after the last major expansion of the NFL. In 2001 the NFL added the Houston Texans to make the league a total of 32 teams. This is also when it realigned so that the teams were arranged in 2 conferences with 4 divisions each. This is the longest period of time that the league has been "stable" (not adding or rearrangeing teams). It is defined as 2002 to the present.

For our analysis we've chosen to use the Modern era to define and filter our data. This will help us remove some of our null values on top of being the most consistent era of play. Most games in the modern era resemble each other in  terms of how the game is played and what players are playing it (which can affect Vegas betting). It will be easier for any model we build as there will, hopefully, be less noise in our data.

We encourage you to take the data and filter it based on your own assumptions though!

In [5]:
#filtering data to just the modern era
modern_era_df <- nfl_df_filtered %>%
    filter(as.numeric(schedule_season) > 2001)

#The elo data is missing the 2018 season and so we'll drop those rows
modern_era_df <- modern_era_df %>% drop_na(home_team_elo, away_team_elo)

#checking nulls
print("Total NULLs in each column for Modern Era data")
colSums(is.na(modern_era_df))

[1] "Total NULLs in each column for Modern Era data"


schedule_date    schedule_season       home_team_id       away_team_id 
                 0                  0                  0                  0 
        home_score         away_score   team_favorite_id    spread_favorite 
                 0                  0                  0                  0 
     home_team_elo      away_team_elo home_team_win_prob 
                 0                  0                  0

## Step 3. Exporting our data

Now that we have our data prepared and cleaned, we'll go ahead and generate a CSV so we don't have to do all those joins again during our model.

In [6]:
#write to csv that lands in the current working directory
write.csv(modern_era_df,file="final_output.csv")